In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import joblib

In [2]:
final_rmse = 5000

In [3]:
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [4]:
def prepare_data(df, sequence_length):
    data = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler

In [10]:
def build_lstm_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(LSTM(units, return_sequences=True))
    model.add(LSTM(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [11]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    
    global final_rmse
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'LSTMmodel_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    gru_loss_history = loss_history.history['loss']
    
    gru_predictions = model.predict(X_test)
    gru_predictions = scaler.inverse_transform(gru_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, gru_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    if rmse <= final_rmse:
        final_rmse = rmse
        joblib.dump(model, "LSTM_model_gru.pkl")
        plt.plot(y_test, label='Actual')
        plt.plot(gru_predictions, label='Predicted')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        plt.savefig(f'BTC_actual_vs_predicted_LSTM.png')
        plt.close()
    
        plt.figure(figsize=(10, 6))
        plt.plot(loss_history.history['loss'], label='Training Loss')
        plt.plot(loss_history.history['val_loss'], label='Validation Loss')
        plt.title('Epoch Loss Curve')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f'BTC_loss_curve_LSTM.png')
        plt.close()
    
    return model, rmse


In [12]:
symbol = 'BTC-USD'
start_date = '2020-01-01'
end_date = '2024-01-31'
epochs_range = [100]
batch_sizes = [4,8]
sequence_lengths = [25,60]
units_range = [50, 100]
layers_range = [2, 3]
results = []

In [13]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
for epochs in epochs_range:
    for batch_size in batch_sizes:
        for sequence_length in sequence_lengths:
            for units in units_range:
                for layers in layers_range:
                    print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
                    X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
                    model = build_lstm_model(X_train, units=units, layers=layers)
                    model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)
                    results.append({'epochs': epochs, 'batch_size': batch_size, 'sequence_length': sequence_length, 'units': units, 'layers': layers, 'rmse': rmse})

Training model with epochs=100, batch_size=4, sequence_length=25, units=50, layers=2

Epoch 1/100

268/268 [==============================] - 17s 31ms/step - loss: 0.0065 - val_loss: 0.0015
Epoch 2/100
  4/268 [..............................] - ETA: 6s - loss: 0.0035

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 7s 25ms/step - loss: 0.0028 - val_loss: 0.0012
Epoch 3/100
268/268 [==============================] - 8s 29ms/step - loss: 0.0029 - val_loss: 4.8882e-04
Epoch 4/100
268/268 [==============================] - 7s 25ms/step - loss: 0.0015 - val_loss: 3.6143e-04
Epoch 5/100
268/268 [==============================] - 7s 25ms/step - loss: 0.0012 - val_loss: 3.8366e-04
Epoch 6/100
268/268 [==============================] - 6s 24ms/step - loss: 0.0012 - val_loss: 2.4269e-04
Epoch 7/100
268/268 [==============================] - 6s 23ms/step - loss: 9.8900e-04 - val_loss: 7.1535e-04
Epoch 8/100
268/268 [==============================] - 6s 23ms/step - loss: 8.7431e-04 - val_loss: 2.4515e-04
Epoch 9/100
268/268 [==============================] - 6s 24ms/step - loss: 7.4708e-04 - val_loss: 2.3296e-04
Epoch 10/100
268/268 [==============================] - 6s 23ms/step - loss: 7.5741e-04 - val_loss: 2.8744e-04
Epoch 11/100
268/268 [=======================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 10s 36ms/step - loss: 0.0039 - val_loss: 8.2013e-04
Epoch 3/100
268/268 [==============================] - 10s 36ms/step - loss: 0.0030 - val_loss: 7.0827e-04
Epoch 4/100
268/268 [==============================] - 10s 37ms/step - loss: 0.0021 - val_loss: 5.9119e-04
Epoch 5/100
268/268 [==============================] - 10s 36ms/step - loss: 0.0016 - val_loss: 6.3342e-04
Epoch 6/100
268/268 [==============================] - 10s 36ms/step - loss: 0.0013 - val_loss: 3.6508e-04
Epoch 7/100
268/268 [==============================] - 10s 36ms/step - loss: 0.0014 - val_loss: 3.0551e-04
Epoch 8/100
268/268 [==============================] - 10s 36ms/step - loss: 9.5321e-04 - val_loss: 5.4217e-04
Epoch 9/100
268/268 [==============================] - 10s 36ms/step - loss: 8.7836e-04 - val_loss: 2.1585e-04
Epoch 10/100
268/268 [==============================] - 10s 36ms/step - loss: 9.1669e-04 - val_loss: 2.2791e-04
Epoch 11/100
268/268 [==============

...

134/134 [==============================] - 7s 52ms/step - loss: 0.0038 - val_loss: 8.5817e-04
Epoch 3/100
134/134 [==============================] - 7s 52ms/step - loss: 0.0024 - val_loss: 5.9544e-04
Epoch 4/100
134/134 [==============================] - 7s 51ms/step - loss: 0.0020 - val_loss: 0.0012
Epoch 5/100
134/134 [==============================] - 7s 52ms/step - loss: 0.0016 - val_loss: 5.3496e-04
Epoch 6/100
134/134 [==============================] - 7s 51ms/step - loss: 0.0014 - val_loss: 7.1218e-04
Epoch 7/100
134/134 [==============================] - 7s 52ms/step - loss: 0.0013 - val_loss: 7.3852e-04
Epoch 8/100
134/134 [==============================] - 7s 52ms/step - loss: 0.0011 - val_loss: 8.3157e-04
Epoch 9/100
134/134 [==============================] - 7s 52ms/step - loss: 0.0011 - val_loss: 3.0239e-04
Epoch 10/100
134/134 [==============================] - 7s 52ms/step - loss: 8.2827e-04 - val_loss: 4.6552e-04
Epoch 11/100
134/134 [==============================] - 7

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 10s 75ms/step - loss: 0.0045 - val_loss: 0.0030
Epoch 3/100
134/134 [==============================] - 10s 77ms/step - loss: 0.0029 - val_loss: 7.1703e-04
Epoch 4/100
134/134 [==============================] - 10s 72ms/step - loss: 0.0027 - val_loss: 0.0014
Epoch 5/100
134/134 [==============================] - 10s 72ms/step - loss: 0.0025 - val_loss: 0.0013
Epoch 6/100
134/134 [==============================] - 10s 72ms/step - loss: 0.0018 - val_loss: 4.5752e-04
Epoch 7/100
134/134 [==============================] - 9s 69ms/step - loss: 0.0015 - val_loss: 7.1960e-04
Epoch 8/100
134/134 [==============================] - 9s 70ms/step - loss: 0.0014 - val_loss: 3.3642e-04
Epoch 9/100
134/134 [==============================] - 9s 70ms/step - loss: 0.0013 - val_loss: 0.0011
Epoch 10/100
134/134 [==============================] - 9s 71ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 11/100
134/134 [==============================] - 9s 70ms/step - l

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 15s 111ms/step - loss: 0.0038 - val_loss: 0.0010
Epoch 3/100
134/134 [==============================] - 15s 115ms/step - loss: 0.0026 - val_loss: 6.7751e-04
Epoch 4/100
134/134 [==============================] - 15s 112ms/step - loss: 0.0021 - val_loss: 8.5901e-04
Epoch 5/100
134/134 [==============================] - 15s 112ms/step - loss: 0.0018 - val_loss: 0.0011
Epoch 6/100
134/134 [==============================] - 15s 113ms/step - loss: 0.0016 - val_loss: 3.6165e-04
Epoch 7/100
134/134 [==============================] - 15s 115ms/step - loss: 0.0012 - val_loss: 5.3545e-04
Epoch 8/100
134/134 [==============================] - 15s 115ms/step - loss: 0.0013 - val_loss: 3.2050e-04
Epoch 9/100
134/134 [==============================] - 15s 114ms/step - loss: 8.2005e-04 - val_loss: 2.8042e-04
Epoch 10/100
134/134 [==============================] - 15s 114ms/step - loss: 8.7787e-04 - val_loss: 2.7763e-04
Epoch 11/100
134/134 [=================